
# DynamicSpatialAttention — Satır Satır, Parametre Parametre İnceleme

Bu notebook, aşağıdaki `DynamicSpatialAttention` modülünün **en ince detayına kadar** açıklamasını içerir.

Modülün temel amacı:
- Girdi feature map üzerinde **nerelere bakılacağını** belirleyen bir **spatial attention maskesi** üretmek.
- Klasik CBAM-Spatial gibi `avg_map` + `max_map` oluşturur.
- Tek bir 7×7 conv yerine **çoklu branch (farklı kernel/dilation)** kullanır.
- Hangi branch’in daha uygun olduğuna **router** karar verir (input’a bağlı softmax ağırlıkları).
- Sonunda `sigmoid/hardsigmoid` ile 0–1 arası maske üretip `x` ile çarpar.

---

## İncelenen Kod


* Spatial Attention’ı kafada oturtan 5 cümle (çekirdek)

* Kanalı ez, konumu çıkar: avg_map ve max_map ile (B,1,H,W) iki harita üret.

* Bunları üst üste koy: s = cat([avg_map, max_map]) → (B,2,H,W)

* K farklı gözle bak: branches ile s’den K tane mask-logit üret → her biri (B,1,H,W)

* Router karar versin: router s’ye bakıp her örnek için rw ağırlıklarını üretir (B,K)

* Karıştır ve uygula: K logiti rw ile ağırlıklı topla, sonra sigmoid(wlogit/T) ile maskeyi üret, x*sa yap.

**Channel attention’da MLP şarttır çünkü kanallar arası ilişkiyi başka türlü öğrenemezsin; spatial attention’da ise uzamsal ilişkiyi zaten convolution öğrendiği için ekstra bir MLP’ye ihtiyaç yoktur.**

---
---
---

## Kod özeti : Genel Tanım 

**Önce parametreleri alıyoruz; kernels normal branch sayısını ve kernel boyutlarını belirliyor, router_hidden ise router içindeki ara kanal genişliği. Init içinde sıcaklık, gate türü ve router_hidden gibi değerler için kontrolleri yapıyoruz. Ardından kernels içindeki değerleri integer’a çevirip çiftse tek yaparak güvenli bir ks listesi oluşturuyoruz. Bu ks üzerinden Conv2d(2→1, k×k) şeklinde normal branch’leri ModuleList içine ekliyoruz; use_dilated=True ise ek olarak bir tane dilated branch (ör. kernel 7, dilation 2) daha branches listesine ekleniyor. Sonra gate fonksiyonunu (sigmoid/hardsigmoid) seçiyoruz. Temperature learnable olacaksa, T=softplus(t_raw)+eps pozitif kalsın diye t_raw’ı nn.Parameter yapıyoruz ve başlangıç değeri temperature olsun diye softplus’ın tersini log(exp(T)-1+eps) formülüyle ayarlıyoruz; learnable değilse sabit temperature’ı register_buffer("T", ...) ile modele bağlıyoruz. Ardından router’ı kuruyoruz: s haritasını önce global average pooling ile (B,2,1,1) özetleyip 1×1 conv + ReLU + 1×1 conv ile (B,K,1,1) logits üretiyoruz; burada K=num_branches. Forward’da önce x üzerinden kanal boyunca ortalama ve maksimum haritaları çıkarıp s=(B,2,H,W) elde ediyoruz. Router s’den logits üretip softmax ile rw=(B,K) branch ağırlıklarını çıkarıyor. Aynı s’yi tüm branch conv’lardan geçirip her biri (B,1,H,W) olacak şekilde K adet maske-logit üretip stack ile z=(B,K,1,H,W) halinde topluyoruz. Sonra rw’yi broadcast edebilmek için (B,K) → (B,K,1,1,1) yapıp rw*z hesaplıyor ve sum(dim=1) ile K branch’i tek haritaya indirerek wlogit=(B,1,H,W) elde ediyoruz. Son olarak wlogit/T ile ölçekleyip gate fonksiyonundan geçirerek sa=(B,1,H,W) spatial attention maskesi üretiyoruz ve y = x * sa ile maskeyi tüm kanallara uyguluyoruz; istenirse debug için rw de döndürülebiliyor.**

---
---
---

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicSpatialAttention(nn.Module):
    def __init__(
        self,
        kernels=(3, 7),
        use_dilated: bool = True,
        dilated_kernel: int = 7,
        dilated_d: int = 2,
        gate: str = "sigmoid",          # "sigmoid" | "hardsigmoid"
        temperature: float = 1.0,
        learnable_temperature: bool = False,
        eps: float = 1e-6,
        router_hidden: int = 8,
        bias: bool = True,
        return_router_weights: bool = False,
    ):
        super().__init__()
        if temperature <= 0:
            raise ValueError("temperature pozitif olmalı.")
        if gate.lower() not in ("sigmoid", "hardsigmoid"):
            raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")
        if router_hidden < 1:
            raise ValueError("router_hidden >= 1 olmalı.")

        self.eps = eps
        self.return_router_weights = return_router_weights

        # aslında burdaki amaç kernelların daha rahat işlemesini ve daha güvenli
        # işlemesini sağlamak.
        # Meseka k tek olmak zorunda ve 1 den büyük olmak zorunda.
        # Eğer bunları sağlıyorsa kernel diye bunları buraya koyabiliyoruz

        ks = []
        for k in kernels:
            k = int(k)
            if k % 2 == 0:
                k += 1
            if k < 1:
                raise ValueError("kernel_size >= 1 olmalı.")
            ks.append(k)

        self.branches = nn.ModuleList()
         ## branches’in yaptığı iş: “K tane farklı yöntemle ‘buraya bak’ haritası üretmek.”
         ## ModuleList = “Bu listedeki katmanlar modelin parçasıdır, unutma.”
        for k in ks:
            p = k // 2
            self.branches.append(nn.Conv2d(2, 1, kernel_size=k, padding=p, bias=False))

        if use_dilated:
            k = int(dilated_kernel)
            if k % 2 == 0:
                k += 1
            if dilated_d < 1:
                raise ValueError("dilated_d >= 1 olmalı.")
            p = dilated_d * (k - 1) // 2
            self.branches.append(
                nn.Conv2d(2, 1, kernel_size=k, padding=p, dilation=dilated_d, bias=False)
            )

        self.num_branches = len(self.branches)

        if gate.lower() == "sigmoid":
            self.gate_fn = torch.sigmoid
        else:
            self.gate_fn = F.hardsigmoid

        self.learnable_temperature = learnable_temperature
        if learnable_temperature:
            t_raw = torch.tensor(float(temperature))
            t_inv = torch.log(torch.exp(t_raw) - 1.0 + eps)
            self.t_raw = nn.Parameter(t_inv) ## “t_raw değerini modelin öğrenebileceği bir ağırlık (parametre) haline getir.”
        else:
            self.register_buffer("T", torch.tensor(float(temperature)))

    # REGİSTER BUFFER İÇERİSİNDEKİ T DEĞERİ ÖYLESİNE BİR DEĞER.YANİ ORAYA A DA YAZABİLİRSİN.AMA O ZAMAN AŞAĞIDAKİ GET_T FONKSİYONUNDAKİ RETURN Ü SELF.A OLARAK DEĞİİTİRMEN GEREKECEK

    # nn.Parameter → öğrenilecek

    # register_buffer → öğrenilmeyecek ama modelle birlikte taşınacak/kaydedilecek

        self.router = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(2, router_hidden, 1, bias=bias),
            nn.ReLU(inplace=True),
            nn.Conv2d(router_hidden, self.num_branches, 1, bias=bias),
        )
    # router_hidden, router’ın karar vermeden önce 2 kanallı bilgiyi biraz işleyip 
    # daha anlamlı hale getirmesini sağlayan ara temsil boyutudur; 
    # bu sayede branch seçimi basit lineer değil, daha esnek ve güçlü olur.

    # “Hangi convolution çıktısı ne kadar dinlenecek?” ağırlığı.

    def _get_T(self) -> torch.Tensor:
        if self.learnable_temperature:
            return F.softplus(self.t_raw) + self.eps
        return self.T

# NEDEN CHANNELS ATTENTİON DA 2 ADET KATMAN VARKEN BURDA HERHANGİ BİR KATMAN YOK ? 
# Channel attention’da kanallar arasında ilişki öğreniyorsun,
# Spatial attention’da ise uzamsal ilişkiyi doğrudan convolution zaten öğreniyor

    def forward(self, x: torch.Tensor):
        avg_map = torch.mean(x, dim=1, keepdim=True) # Her (H,W) konumu için kanalları ortalayıp tek harita çıkarıyor
        max_map, _ = torch.max(x, dim=1, keepdim=True) # “Bu konumda en güçlü sinyal hangi kanaldan gelirse gelsin, ne kadar güçlü?
        s = torch.cat([avg_map, max_map], dim=1)  # (B,2,H,W)

        logits = self.router(s).flatten(1)             # (B,K)
        rw = torch.softmax(logits, dim=1)              # (B,K) # Örn: rw[b] = [0.6, 0.3, 0.1]

# logits aslında router’dan çıkan ve softmax’a girecek ham skorlar; 
# router, s=(B,2,H,W) haritasını önce global havuzlama ile (B,2,1,1) özetine indiriyor,
# ardından 1×1 katmanlarla bunu (B,K,1,1) olacak şekilde num_branches=K tane skora 
# çeviriyor;
# bu skorlar flatten ile (B,K) yapılıyor ve softmax ile rw elde ediliyor, 
# böylece model her örnek için 
# “hangi kernel/branch daha iyi?” sorusuna yüzde olarak cevap veriyor.

        # “Her branch conv’u al, aynı s input’una uygula.” Elde ettiğin şey:K tane ayrı ayrı (B,1,H,W) harita
        z = torch.stack([br(s) for br in self.branches], dim=1)  # (B,K,1,H,W) # Örn: z[b,0] = 3×3 conv’un maskesi, z[b,1] = 7×7 maskesi, ...
        # z, branches içindeki farklı kernel/dilation’a sahip conv katmanlarının
        # her birinin s üzerinden ürettiği spatial logit haritalarının, 
        # branch boyutunda bir araya getirilmiş halidir.


        wlogit = (rw[:, :, None, None, None] * z).sum(dim=1)     # (B,1,H,W)
        #    rw şu an (B,K).
        # Ama z ile çarpabilmek için rw’yi şu şekle getirmeliyiz
        # (B,K) → (B,K,1,1,1) , Bu “None” eklemek şu demek:
        # “Bu boyutlarda tek değer var, otomatik yay.”
        #         rw_expanded * z
        # rw_expanded: (B,K,1,1,1)
        # z: (B,K,1,H,W)
        # Sonuç:
        # (B,K,1,H,W)
        # .sum(dim=1)
        # şunu yapar:
        # “Branch boyutunu topla, K haritayı tek haritaya indir.”
        # ===== rw * z her branch maskesini kendi ağırlığıyla ölçekler; 
        # .sum(dim=1) ise K branch’i 
        # tek bir (B,1,H,W) haritaya indirerek nihai spatial logit haritasını üretir.

        T = self._get_T()
        sa = self.gate_fn(wlogit / T)  # (B,1,H,W)


        y = x * sa

#         “Neden çarptık?” (y = x * sa)
# Çünkü sa dediğin şey dikkat maskesi.

# sa shape: (B, 1, H, W)
# x shape: (B, C, H, W)

# Çarpınca ne oluyor?
# (B,1,H,W) maskesi tüm kanallara broadcast edilir.
# Her (h,w) konumu için:
# sa büyükse → o konumdaki tüm kanallar güçlenir / korunur
# sa küçükse → o konumdaki tüm kanallar kısılır / bastırılır

# Yani bu çarpma şunu yapıyor:
# “Önemli yerleri geçir, önemsiz yerleri sustur.”

# Bu bir gating (kapı) mekanizması.
# Add (toplama) yapsaydın “bastırma” olmazdı; sadece bir şey eklemiş olurduk.
# Attention’ın klasik uygulaması bu yüzden çarpmadır.

        if self.return_router_weights:
            return y, sa, rw
        return y, sa



---
# 1) Modül Ne Üretiyor? (Çıktılar ve Şekiller)

Girdi:
- `x`: `(B, C, H, W)`

Çıktı:
- `y`: `(B, C, H, W)`  → attention uygulanmış çıktı
- `sa`: `(B, 1, H, W)` → spatial attention maskesi
- opsiyonel `rw`: `(B, K)` → router branch ağırlıkları (her örnek için)

Burada `K = num_branches`:
- `kernels` listesindeki branch sayısı
- `use_dilated=True` ise +1 dilated branch

Örnek:
- `kernels=(3,7)` ve `use_dilated=True` → K=3



---
# 2) Parametreler: Ne İşe Yarar?

## `kernels=(3,7)`
- Normal branch’lerde kullanılacak kernel boyutları.
- Her kernel için ayrı bir `Conv2d(2→1)` branch kurulur.
- İçeride **tek sayıya zorlanır** (çiftse +1 yapılır).

## `use_dilated=True`
- Ek bir **dilated** branch ekler.

## `dilated_kernel=7`, `dilated_d=2`
- Dilated conv’un kernel boyutu ve dilation oranı.
- Dilation, receptive field’ı büyütür ama parametre sayısını artırmaz.

## `gate="sigmoid"` / `"hardsigmoid"`
- En sonda maskeyi 0–1 aralığına sıkıştırır.

## `temperature=1.0`, `learnable_temperature=False`, `eps=1e-6`
- `wlogit / T` ile maskenin keskinliği ayarlanır.
- Learnable ise `T=softplus(t_raw)+eps` ile pozitif kalması garanti edilir.

## `router_hidden=8`
- Router’ın iç hidden kanal sayısı.

## `bias=True`
- Router içindeki 1×1 conv’larda bias kullanımı.

## `return_router_weights=False`
- Debug için `rw` döndürür.



---
# 3) Kernel Düzeltmeleri: Çift Kernel Neden Tek Yapılıyor?

Kod:
```python
if k % 2 == 0:
    k += 1
p = k // 2
```

- Tek kernel ile padding simetrik olur.
- `padding = k//2` seçilince `H,W` korunur.



---
# 4) Branch’ler: `Conv2d(2→1)` Neden 2 Giriş Kanalı?

Forward’da:
- `avg_map`: `(B,1,H,W)`
- `max_map`: `(B,1,H,W)`
- `s = cat([avg_map, max_map], dim=1)` → `(B,2,H,W)`

Branch conv’lar `s`’yi alır ve `(B,1,H,W)` logit haritası üretir.



---
# 5) Dilated Branch ve Padding Formülü

Dilated conv’da efektif alan büyür.
Boyutu korumak için:

```python
p = d * (k - 1) // 2
```



---
# 6) Router: Input’a Göre Branch Ağırlığı Üreten Küçük Ağ

Router:
- `AdaptiveAvgPool2d(1)` ile `(B,2,H,W)` → `(B,2,1,1)`
- 1×1 conv + ReLU ile küçük bir MLP gibi çalışır
- Son 1×1 conv: `(B, router_hidden,1,1)` → `(B,K,1,1)`

Sonra:
- `flatten(1)` → `(B,K)`
- `softmax(dim=1)` → `(B,K)` (her örnek için olasılık dağılımı)



---
# 7) Forward: En Önemli Satırlar

## 7.1) `s` üretimi
```python
avg_map = mean(x, dim=1, keepdim=True)   # (B,1,H,W)
max_map, _ = max(x, dim=1, keepdim=True) # (B,1,H,W)
s = cat([avg_map, max_map], dim=1)       # (B,2,H,W)
```

## 7.2) Router ağırlıkları
```python
logits = router(s).flatten(1)  # (B,K)
rw = softmax(logits, dim=1)    # (B,K)
```

## 7.3) Branch çıktıları + ağırlıklı toplam
```python
z = stack([br(s) for br in branches], dim=1)  # (B,K,1,H,W)
wlogit = (rw[:, :, None, None, None] * z).sum(dim=1)  # (B,1,H,W)
```

Buradaki `None` eklemeleri sadece broadcast içindir:
- `(B,K)` → `(B,K,1,1,1)`
- `(B,K,1,H,W)` ile çarpılabilir hale gelir.

## 7.4) Temperature + gate + uygulama
```python
sa = gate_fn(wlogit / T)  # (B,1,H,W)
y = x * sa                # (B,C,H,W)
```



---
# 8) Mini Deneyler (Şekil ve Router Ağırlıkları)


In [3]:
# Şekil kontrolü ve router weights örneği
torch.manual_seed(0)
x = torch.randn(2, 64, 56, 56)

m = DynamicSpatialAttention(kernels=(3,7), use_dilated=True, return_router_weights=True)
y, sa, rw = m(x)

print("y:", y.shape)
print("sa:", sa.shape)
print("rw:", rw.shape, "num_branches:", m.num_branches)
print("rw[0]:", rw[0])


y: torch.Size([2, 64, 56, 56])
sa: torch.Size([2, 1, 56, 56])
rw: torch.Size([2, 3]) num_branches: 3
rw[0]: tensor([0.4861, 0.2412, 0.2727], grad_fn=<SelectBackward0>)


In [4]:

# Temperature etkisi: maskenin istatistiği
def stats(T):
    m = DynamicSpatialAttention(temperature=T, learnable_temperature=False)
    y, sa = m(x)
    return (T, float(sa.mean()), float(sa.std()), float(sa.min()), float(sa.max()))

for T in [0.5, 1.0, 2.0, 5.0]:
    print("T, mean, std, min, max =", stats(T))


T, mean, std, min, max = (0.5, 0.5809781551361084, 0.06028931215405464, 0.28509071469306946, 0.7919651865959167)
T, mean, std, min, max = (1.0, 0.4193452000617981, 0.05250205472111702, 0.2961040139198303, 0.6760010123252869)
T, mean, std, min, max = (2.0, 0.46876415610313416, 0.017804302275180817, 0.4033283293247223, 0.5379263162612915)
T, mean, std, min, max = (5.0, 0.5082592368125916, 0.011276109144091606, 0.46128103137016296, 0.5573007464408875)


C:\Users\hdgn5\AppData\Local\Temp\ipykernel_22988\3387644643.py:5: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  return (T, float(sa.mean()), float(sa.std()), float(sa.min()), float(sa.max()))
